In [2]:
from transformers import GPT2Tokenizer
!pip install -U transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 1.8 MB/s eta 0:00:0000:010:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 1.3 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 498.6 kB/s eta 0:00:0000:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 1.4 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 1.6 MB/s eta 0:00:00-:--:--

[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

/Users/apple/PycharmProjects/inference-with-huggingface/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Load pretrained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [3]:
text = "fuck you,"
indexed_tokens = tokenizer.encode(text)

In [4]:
token_tensor = torch.tensor([indexed_tokens])

In [5]:
print(indexed_tokens)

[31699, 345, 11]


In [6]:
#Load pretrained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

Loading weights: 100%|██████████| 148/148 [00:00<00:00, 537.56it/s, Materializing param=transformer.wte.weight]             
GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
#No backpropagation
with torch.no_grad():
    outputs = model(token_tensor)
    predictions = outputs[0]


In [8]:
#Get the predict next subword
predict_index = torch.argmax(predictions[0, -1, :]).item()
predict_text = tokenizer.decode(indexed_tokens + [predict_index])
print(predict_text)

fuck you, I


In [9]:
# Your prompt
prompt = "I don't know, what should I do"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate continuation (sentence)
# max_new_tokens controls how long the generated text will be
output_ids = model.generate(
    **inputs,
    max_new_tokens=50,   # adjust for longer sentence
    do_sample=True,      # enables randomness (otherwise argmax)
    top_k=50,            # sample from top 50 tokens
    top_p=0.98,          # nucleus sampling
    temperature=0.8      # randomness factor
)

# Decode generated text
generated_text = tokenizer.decode(output_ids[0])
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I don't know, what should I do. I need to get a job."

"Okay, you have to get a job, and I'm going to need to get a job for my son."

"I'm good with kids, if I want to be with my
